# Beginner's Tutorial
You may run the notebook as is if you have the sample data file, but you must change the 'file_dir' variable in block 5 to have the correct file path. Try changing the model, variable type, variable names, plotting values, and so on to interact with the data. All of the model readers have identical syntax and function, so you can choose any model reader and expect the same results. Execution time varies between datasets.
Sample datasets may be obtained from https://ccmc.gsfc.nasa.gov/RoR_WWW/output_files/KAMODO_DEMO/. For examples of other functions, such as the flythrough, see the other notebooks in this directory.

In [ ]:
# Show list of models currently available.
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Choose_Model('')

In [ ]:
# Access documentation.
help(MW.Variable_Search)

In [ ]:
# Show all the possible variables
MW.Variable_Search('')

In [ ]:
# Find what model has the variable.
MW.Variable_Search('temperature')

In [ ]:
# The same output, but only for the requested model.
model = 'CTIPe'
MW.Variable_Search('temperature', model=model)

In [ ]:
# But I just want to see the variables from CTIPe...
MW.Variable_Search('', model=model)

### IMPORTANT!
Change the file_dir variable in the next cell to be the file path to the chosen dataset before running.

In [ ]:
# That's a lot, but what is in my data?
file_dir = 'C:/Users/rringuet/Kamodo_Data/CTIPe/Storm_201303/'
# This command and any call to MW.Variable_Search that includes the file_dir
# also performs any file conversion or pre-processing necessary.
MW.Variable_Search('', model=model, file_dir=file_dir)

In [ ]:
# Find available temperature variables in the chosen CTIPe data.
MW.Variable_Search('temperature', model=model, file_dir=file_dir)

In [ ]:
# What are the time ranges available in my data?
MW.File_Times(model, file_dir)

# IMPORTANT!
- If you decide to add or remove data from the chosen file directory, you must remove the TIEGCM_list.txt and TIEGCM_times.txt files from the same directory and rerun the above command. The name changes with the model name (e.g. WACCMX_list.txt and WACCMX_times.txt for the WACCMX model).
- The rest of the notebook shows how to use the model readers. If your interest is in a different function, please move on to the notebook of your choice.

In [ ]:
# Import the correct model reader object.
reader = MW.Model_Reader(model)

In [ ]:
# Documentation for all the readers is the same.
help(reader)

In [ ]:
# Access the model data for the chosen model.
# This default function call functionalizes all the variables in the data,
# including gridded versions for visualization.
# TIEGCM has a data type error due to a file creation error.
kamodo_object_default = reader(file_dir)  # (gridded_int=True is default)
kamodo_object_default

In [ ]:
# Note the arg_units column of each variable shows the coordinate names
# and units even if this information is not indicated in the function
# signature.
kamodo_object_default.detail()

In [ ]:
# Option 1: functionalize all variables, excluding gridded versions.
kamodo_object_1 = reader(file_dir, gridded_int=False)
kamodo_object_1

In [ ]:
# Option 2: load a single or multiple variables with the gridded versions.
# This is the most common method to functionalize the desired data.
kamodo_object_2 = reader(file_dir, variables_requested=['T_n', 'T_i'])
kamodo_object_2

## Note:
- The T_n variable as found in the data depends on pressure level (T_n_ilev). An alternate version of the variable was automatically created to depend on height (T_n) by inverting the height variable (H_ilev1), which was also automatically retrieved.
- The T_i variable already depends on height, so no inversion was needed.
- Requesting variables that are NOT in the list above triggers a printed message: '... name not recognized'.
- Requesting variables that ARE in the list above but not in the file triggers a printed message: '...not available'.

In [ ]:
# Option 3: load a single or multiple variables without gridded version.
# This is what is used in Kamodo's flythrough.
kamodo_object_3 = reader(file_dir, variables_requested=['T_n', 'T_i'], gridded_int=False)
kamodo_object_3

In [ ]:
# Show datetime object for midnight of the first day of the dataset.
kamodo_object_default.filedate

In [ ]:
# Print max and min of coordinate grids. Coordinate units are shown in the above blocks.
from kamodo import get_defaults
for var in ['T_n', 'T_i']:
    print('\n', var)
    defaults_ijk = get_defaults(kamodo_object_2[var+'_ijk'])
    for key in defaults_ijk:
        print(key, len(defaults_ijk[key]), defaults_ijk[key].min(), defaults_ijk[key].max())

In [ ]:
# An example of Kamodo's default 2D plot.
# The undefined areas of the plot are due to the changing height corresponding to the 
# pressure level that the original neutral temperature variable depends on. This is normal.
kamodo_object_default.plot('T_n_ijk', plot_partial={'T_n_ijk': {'time': 22.5, 'height': 350.}})

In [ ]:
# Nicer version (coming soon as an option with simpler code).
fig = kamodo_object_default.plot('T_n_ijk', plot_partial={'T_n_ijk': {'time': 22.5, 'height': 350.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Example of a 1D plot with more than one variable.
kamodo_object_default.plot('T_n_ijk', 'T_i_ijk',
                           plot_partial={'T_n_ijk': {'time': 22.5, 'lon': 2.5, 'height': 350.},
                                         'T_i_ijk': {'time': 22.5, 'lon': 2.5, 'height': 350.}})

In [ ]:
# Example of interpolation command for two points for a regular function.
# This is the method used in Kamodo's flythrough
kamodo_object_default['T_n']([[22.5, -20.5, -20.5, 350.], [22.7, -20.5, -20.5, 350.]])

In [ ]:
# Example of interpolation command for a gridded function.
# The value will be the same two values as the previous block's result.
# The gridded functions slice through the data at the values given.
kamodo_object_default['T_n_ijk'](time=[22.5, 22.7], lon=-20.5, lat=-20.5, height=350.)